This notebook can be used to sample the json files that serve as base for the hackaton

In [ ]:
import pandas as pd
import json
import numpy as np
from random import choice

In [ ]:
def generate_subsequences_of_random_length(l, n, m, p):
    """ Takes the list and generates subsequences of random length containing an element from l. 
    
    The length of the subsequence is Binomial(m, p) distributed.
    
    Args:
        l (list): the list with the elements of the subsequence
        n (int): the size of the subsequence
        m (int): the maximal size of subsequence (Binomial(m, p) distributed)
        p (float): probability of the length of subsequence (Binomial(m, p) distributed)
    
    Will return the concatenated subsequences as list of size l.
    """
    result = []
    while len(result) < n:
        result += [choice(l)] * int(np.random.binomial(m, p, 1))
                   
    return result[:n]
    

In [ ]:
r = generate_subsequences_of_random_length(range(5), 30, 5, 0.5)
np.array(r)

# Prod schedule

In [ ]:
n_total = 100
n_seq = 8


grade = generate_subsequences_of_random_length(["STI1", "ST2", "PPQ"], n_total, 10, 0.3)


df_prod_sced = pd.DataFrame({"heat_sq": np.repeat(range(1, n_total//n_seq + 2), n_seq)[:n_total], 
                             "head_id": [f"heat-{i}" for i in range(n_total)],
                             "steel_grade": grade,
                             "required_weight": np.random.binomial(2000, 0.5, size=n_total),
                             "chemistry": [{"cu_pct": choice([0.08, 0.1, 0.12])} for _ in range(n_total)]
                            })
df_prod_sced

In [ ]:
with open('production_schedule2.json', 'w') as outfile:
    json.dump(df_prod_sced.to_dict(orient='record'), outfile)

# Inventory

In [ ]:
df_inventory = pd.DataFrame({"scrap_type":["bushling", "pig_iron"], 
                             "required_weight": np.random.binomial(10000, 0.7, size=2),
                             "status": "on_hand"
                            })
df_inventory

In [ ]:
with open('srap_inventory2.json', 'w') as outfile:
    json.dump(df_inventory.to_dict(orient='record'), outfile)

# Orders

In [ ]:
df_order = pd.DataFrame({"scrap_type": ["bushling", "pig_iron"], 
                         "order_date":["20190405", "20190406"], 
                         "status": "delivered", 
                         "weight": np.random.binomial(10000, 0.7, size=2), 
                         "price_per_ton": np.random.binomial(200, 0.7, size=2)
                        })
df_order

In [ ]:
with open('srap_orders2.json', 'w') as outfile:
    json.dump(df_order.to_dict(orient='record'), outfile)